In [1]:
pip install --upgrade py2neo

Requirement already up-to-date: py2neo in c:\users\rakesh\anaconda3\lib\site-packages (2021.2.3)
Note: you may need to restart the kernel to use updated packages.


In [2]:
 pip install --upgrade --pre py2neo

Requirement already up-to-date: py2neo in c:\users\rakesh\anaconda3\lib\site-packages (2021.2.3)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from py2neo import Graph,Node,Relationship

In [5]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "rakesh"))

In [18]:
!pip install neo4j
from neo4j import GraphDatabase
import neo4jupyter
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "rakesh"))

In [9]:
query = "MATCH (n) return n"
graph.run(query).data()

[{'n': Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862')},
 {'n': Node('Movie', id='2', title='Jumanji (1995)', tmdbId='8844')},
 {'n': Node('Movie', id='3', title='Grumpier Old Men (1995)', tmdbId='15602')},
 {'n': Node('Movie', id='4', title='Waiting to Exhale (1995)', tmdbId='31357')},
 {'n': Node('Movie', id='5', title='Father of the Bride Part II (1995)', tmdbId='11862')},
 {'n': Node('Movie', id='6', title='Heat (1995)', tmdbId='949')},
 {'n': Node('Movie', id='7', title='Sabrina (1995)', tmdbId='11860')},
 {'n': Node('Movie', id='8', title='Tom and Huck (1995)', tmdbId='45325')},
 {'n': Node('Movie', id='9', title='Sudden Death (1995)', tmdbId='9091')},
 {'n': Node('Movie', id='10', title='GoldenEye (1995)', tmdbId='710')},
 {'n': Node('Movie', id='11', title='American President, The (1995)', tmdbId='9087')},
 {'n': Node('Movie', id='12', title='Dracula: Dead and Loving It (1995)', tmdbId='12110')},
 {'n': Node('Movie', id='13', title='Balto (1995)', tmdbId='21032')}

In [10]:
query="Match (n)-[r]->(m) return n,r,m"
graph.run(query).data()

[{'n': Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'),
  'r': IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Adventure')),
  'm': Node('Genre', name='Adventure')},
 {'n': Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'),
  'r': IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Animation')),
  'm': Node('Genre', name='Animation')},
 {'n': Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'),
  'r': IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Children')),
  'm': Node('Genre', name='Children')},
 {'n': Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'),
  'r': IS_GENRE(Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'), Node('Genre', name='Comedy')),
  'm': Node('Genre', name='Comedy')},
 {'n': Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'),
  'r': IS_GENRE(Node('Movie', id='1', titl

In [11]:
query="""LOAD CSV WITH HEADERS FROM 'file:///movies.csv' AS line
 MERGE (m:Movie{ id:line.movieId, title:line.title}) 
 FOREACH (gName in split(line.genres, '|') | 
    MERGE (g:Genre {name:gName}) 
    MERGE (m)-[:IS_GENRE]->(g)
)"""
graph.run(query).data()

[]

In [12]:
query="""Match (n) return n """
graph.run(query).data()

[{'n': Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862')},
 {'n': Node('Movie', id='2', title='Jumanji (1995)', tmdbId='8844')},
 {'n': Node('Movie', id='3', title='Grumpier Old Men (1995)', tmdbId='15602')},
 {'n': Node('Movie', id='4', title='Waiting to Exhale (1995)', tmdbId='31357')},
 {'n': Node('Movie', id='5', title='Father of the Bride Part II (1995)', tmdbId='11862')},
 {'n': Node('Movie', id='6', title='Heat (1995)', tmdbId='949')},
 {'n': Node('Movie', id='7', title='Sabrina (1995)', tmdbId='11860')},
 {'n': Node('Movie', id='8', title='Tom and Huck (1995)', tmdbId='45325')},
 {'n': Node('Movie', id='9', title='Sudden Death (1995)', tmdbId='9091')},
 {'n': Node('Movie', id='10', title='GoldenEye (1995)', tmdbId='710')},
 {'n': Node('Movie', id='11', title='American President, The (1995)', tmdbId='9087')},
 {'n': Node('Movie', id='12', title='Dracula: Dead and Loving It (1995)', tmdbId='12110')},
 {'n': Node('Movie', id='13', title='Balto (1995)', tmdbId='21032')}

In [13]:
query="""LOAD CSV WITH HEADERS FROM "file:///ratings.csv" AS line
 MATCH (m:Movie {id:line.movieId})
 MERGE (u:User {id:line.userId})
 MERGE (u)-[:RATED { rating: toFloat(line.rating)}]->(m);"""
graph.run(query).data()

[]

In [19]:
query="""LOAD CSV WITH HEADERS FROM "file:///tags.csv" AS line
 MATCH (m:Movie {id:line.movieId})
 MATCH (u:User {id:line.userId})
 CREATE (u)-[:TAGGED { tag: line.tag}]->(m);"""
graph.run(query).data()

[]

In [51]:
(neo4jupyter.draw(graph,{'tag':'movieId'}))


In [21]:
query="""LOAD CSV WITH HEADERS FROM "file:///links.csv" AS line
MATCH (m:Movie {id:line.movieId})
SET m.tmdbId=line.tmdbId;"""
graph.run(query).data()

[]

In [22]:
query="""LOAD CSV WITH HEADERS FROM "file:///directors.csv" AS line
MATCH (m:Movie{ tmdbId:line.movieId})
MERGE (p:Person{name:line.person_name})
MERGE (p)-[:DIRECTED]->(m);"""
graph.run(query).data()

[]

In [23]:
query="""LOAD CSV WITH HEADERS FROM "file:///roles.csv" AS line
MATCH (m:Movie{ tmdbId:line.movieId})
MERGE (p:Person{name:line.person_name})
CREATE (p)-[r:ACTED_IN] ->(m)
SET  r.role= line.role;"""
graph.run(query).data()

[]

In [25]:
query="""MATCH (m:Movie {title: "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)"})-[:ACTED_IN|:IS_GENRE|:DIRECTED]-(p)
RETURN m, p"""
graph.run(query).data()
#neo4jupyter.draw(graph,{query})

[{'m': Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'),
  'p': Node('Genre', name='Adventure')},
 {'m': Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'),
  'p': Node('Genre', name='Children')},
 {'m': Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'),
  'p': Node('Genre', name='Fantasy')},
 {'m': Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'),
  'p': Node('Person', name='Chris Columbus')},
 {'m': Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'),
  'p': Node('Person', name='James Phelps')},
 {'m': Node('Movie', id='4896', t

In [27]:
query="""MATCH q=(m:Movie {title: "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)"})-[:ACTED_IN |DIRECTED*..2]-(p)
RETURN q LIMIT 50"""
graph.run(query).data()

[{'q': Path(Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'), DIRECTED(Node('Person', name='Chris Columbus'), Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671')))},
 {'q': Path(Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'), DIRECTED(Node('Person', name='Chris Columbus'), Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671')), DIRECTED(Node('Person', name='Chris Columbus'), Node('Movie', id='40826', title='Rent (2005)', tmdbId='1833')))},
 {'q': Path(Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'), DIRECTED(Node('Person', name='

In [28]:
query="""MATCH (m:Movie {title: "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)"})-[:RATED|:TAGGED]-(u)
RETURN m, u LIMIT 25"""
graph.run(query).data()

[{'m': Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'),
  'u': Node('User', id='387')},
 {'m': Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'),
  'u': Node('User', id='610')},
 {'m': Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'),
  'u': Node('User', id='167')},
 {'m': Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'),
  'u': Node('User', id='380')},
 {'m': Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", tmdbId='671'),
  'u': Node('User', id='382')},
 {'m': Node('Movie', id='4896', title="Harry Potter and the Sorcerer's Stone (a.k.a. 

In [31]:
query="""MATCH (me:User{id:'220'})-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other:User)-[r3:RATED]->(m2:Movie)
WHERE r1.rating > 3 AND r2.rating > 3 AND r3.rating > 3 AND NOT (me)-[:RATED]->(m2)
RETURN distinct m2 AS recommended_movie, count(*) AS score
ORDER BY score DESC
LIMIT 15"""
graph.run(query).data()

[{'recommended_movie': Node('Movie', id='593', title='Silence of the Lambs, The (1991)', tmdbId='274'),
  'score': 7203},
 {'recommended_movie': Node('Movie', id='4993', title='Lord of the Rings: The Fellowship of the Ring, The (2001)', tmdbId='120'),
  'score': 6563},
 {'recommended_movie': Node('Movie', id='2858', title='American Beauty (1999)', tmdbId='14'),
  'score': 6227},
 {'recommended_movie': Node('Movie', id='110', title='Braveheart (1995)', tmdbId='197'),
  'score': 5894},
 {'recommended_movie': Node('Movie', id='3578', title='Gladiator (2000)', tmdbId='98'),
  'score': 5777},
 {'recommended_movie': Node('Movie', id='527', title="Schindler's List (1993)", tmdbId='424'),
  'score': 5663},
 {'recommended_movie': Node('Movie', id='1136', title='Monty Python and the Holy Grail (1975)', tmdbId='762'),
  'score': 5377},
 {'recommended_movie': Node('Movie', id='4963', title="Ocean's Eleven (2001)", tmdbId='161'),
  'score': 5011},
 {'recommended_movie': Node('Movie', id='1214', tit

In [32]:
query="""MATCH (me:User{id:'220'})-[r:RATED]-(m)
WITH me, avg(r.rating) AS average
MATCH (me)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other:User)-[r3:RATED]->(m2:Movie)
WHERE r1.rating > average AND r2.rating > average AND r3.rating > average AND NOT (me)-[:RATED]->(m2)
RETURN distinct m2 AS recommended_movie, count(*) AS score
ORDER BY score DESC
LIMIT 15"""
graph.run(query).data()

[{'recommended_movie': Node('Movie', id='593', title='Silence of the Lambs, The (1991)', tmdbId='274'),
  'score': 5322},
 {'recommended_movie': Node('Movie', id='4993', title='Lord of the Rings: The Fellowship of the Ring, The (2001)', tmdbId='120'),
  'score': 4276},
 {'recommended_movie': Node('Movie', id='2858', title='American Beauty (1999)', tmdbId='14'),
  'score': 4129},
 {'recommended_movie': Node('Movie', id='527', title="Schindler's List (1993)", tmdbId='424'),
  'score': 4086},
 {'recommended_movie': Node('Movie', id='110', title='Braveheart (1995)', tmdbId='197'),
  'score': 3982},
 {'recommended_movie': Node('Movie', id='3578', title='Gladiator (2000)', tmdbId='98'),
  'score': 3537},
 {'recommended_movie': Node('Movie', id='1214', title='Alien (1979)', tmdbId='348'),
  'score': 3502},
 {'recommended_movie': Node('Movie', id='1136', title='Monty Python and the Holy Grail (1975)', tmdbId='762'),
  'score': 3408},
 {'recommended_movie': Node('Movie', id='1221', title='Godfa

In [33]:
query="""MATCH (me:User{id:'220'})-[r:RATED]-(m)
WITH me, avg(r.rating) AS average
MATCH (me)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other:User)-[r3:RATED]->(m2:Movie)
WHERE r1.rating > average AND r2.rating > average AND r3.rating > average AND NOT (me)-[:RATED]->(m2)
RETURN distinct m2, other, me, m AS recommended_movie, count(m2) AS score
ORDER BY score DESC
LIMIT 20"""
graph.run(query).data()

[{'m2': Node('Movie', id='2', title='Jumanji (1995)', tmdbId='8844'),
  'other': Node('User', id='476'),
  'me': Node('User', id='220'),
  'recommended_movie': Node('Movie', id='318', title='Shawshank Redemption, The (1994)', tmdbId='278'),
  'score': 1},
 {'m2': Node('Movie', id='2', title='Jumanji (1995)', tmdbId='8844'),
  'other': Node('User', id='476'),
  'me': Node('User', id='220'),
  'recommended_movie': Node('Movie', id='32', title='Twelve Monkeys (a.k.a. 12 Monkeys) (1995)', tmdbId='63'),
  'score': 1},
 {'m2': Node('Movie', id='2', title='Jumanji (1995)', tmdbId='8844'),
  'other': Node('User', id='476'),
  'me': Node('User', id='220'),
  'recommended_movie': Node('Movie', id='589', title='Terminator 2: Judgment Day (1991)', tmdbId='280'),
  'score': 1},
 {'m2': Node('Movie', id='2', title='Jumanji (1995)', tmdbId='8844'),
  'other': Node('User', id='476'),
  'me': Node('User', id='220'),
  'recommended_movie': Node('Movie', id='1', title='Toy Story (1995)', tmdbId='862'),
 

In [34]:
query="""MATCH (me:User{id:'318'})-[r:RATED]-(m)
WITH me, avg(r.rating) AS average
MATCH (me)-[t1:TAGGED]->(m:Movie)-[r:RATED]-(me)
MATCH (other:User)-[t2:TAGGED]->(m1:Movie)
WHERE r.rating > average AND t1.tag=t2.tag AND  NOT (me)-[:TAGGED]->(m1) AND  NOT (me)-[:RATED]->(m1)
RETURN m1, other"""
graph.run(query).data()

[{'m1': Node('Movie', id='6564', title='Lara Croft Tomb Raider: The Cradle of Life (2003)', tmdbId='1996'),
  'other': Node('User', id='62')},
 {'m1': Node('Movie', id='6564', title='Lara Croft Tomb Raider: The Cradle of Life (2003)', tmdbId='1996'),
  'other': Node('User', id='62')},
 {'m1': Node('Movie', id='6534', title='Hulk (2003)', tmdbId='1927'),
  'other': Node('User', id='62')},
 {'m1': Node('Movie', id='6534', title='Hulk (2003)', tmdbId='1927'),
  'other': Node('User', id='62')},
 {'m1': Node('Movie', id='96861', title='Taken 2 (2012)', tmdbId='82675'),
  'other': Node('User', id='62')},
 {'m1': Node('Movie', id='96861', title='Taken 2 (2012)', tmdbId='82675'),
  'other': Node('User', id='62')},
 {'m1': Node('Movie', id='27706', title="Lemony Snicket's A Series of Unfortunate Events (2004)", tmdbId='11774'),
  'other': Node('User', id='62')},
 {'m1': Node('Movie', id='27706', title="Lemony Snicket's A Series of Unfortunate Events (2004)", tmdbId='11774'),
  'other': Node('Us

In [36]:
query="""MATCH (me:User{id:'318'})-[r:RATED]-(m:Movie)
WITH me, avg(r.rating) AS average
MATCH (me)-[r:RATED]->(m:Movie)-[:ACTED_IN]-(p:Person)
WHERE r.rating > average
RETURN  p as actor, COUNT(*) AS score 
ORDER BY score DESC LIMIT 10"""
graph.run(query).data()


[{'actor': Node('Person', name='Johnny Depp'), 'score': 20},
 {'actor': Node('Person', name='Matt Damon'), 'score': 20},
 {'actor': Node('Person', name='George Clooney'), 'score': 18},
 {'actor': Node('Person', name='Steve Buscemi'), 'score': 16},
 {'actor': Node('Person', name='Bill Hader'), 'score': 16},
 {'actor': Node('Person', name='Brad Pitt'), 'score': 16},
 {'actor': Node('Person', name='Philip Seymour Hoffman'), 'score': 14},
 {'actor': Node('Person', name='Sean Penn'), 'score': 14},
 {'actor': Node('Person', name='John C. Reilly'), 'score': 14},
 {'actor': Node('Person', name='Leonardo DiCaprio'), 'score': 12}]

In [37]:
query="""MATCH (me:User{id:'318'})-[r:RATED]-(m:Movie)
WITH me, avg(r.rating) AS average
MATCH (me)-[r:RATED]->(m:Movie)-[:DIRECTED]-(p:Person)
WHERE r.rating > average
RETURN  p as director, COUNT(*) AS score 
ORDER BY score DESC LIMIT 10"""
graph.run(query).data()

[{'director': Node('Person', name='Joel Coen'), 'score': 7},
 {'director': Node('Person', name='Christopher Nolan'), 'score': 5},
 {'director': Node('Person', name='Steven Spielberg'), 'score': 5},
 {'director': Node('Person', name='Quentin Tarantino'), 'score': 4},
 {'director': Node('Person', name='Kevin Smith'), 'score': 4},
 {'director': Node('Person', name='Tim Burton'), 'score': 3},
 {'director': Node('Person', name='Wes Anderson'), 'score': 3},
 {'director': Node('Person', name='Edgar Wright'), 'score': 3},
 {'director': Node('Person', name='Larry Charles'), 'score': 3},
 {'director': Node('Person', name='Paul Thomas Anderson'), 'score': 3}]

In [39]:
query="""MATCH (me:User{id:'318'})-[r:RATED]-(m:Movie)
WITH me, avg(r.rating) AS average
MATCH (me)-[r:RATED]->(m:Movie)-[:IS_GENRE]-(p:Genre)
WHERE r.rating > average
RETURN  p.title as genre, COUNT(*) AS score 
ORDER BY score DESC LIMIT 10"""
graph.run(query).data()

[{'genre': None, 'score': 1066}]

In [40]:
query="""MATCH (me:User{id:'318'})-[r:RATED]-(m:Movie)
WITH me, avg(r.rating) AS average
MATCH (me)-[r:RATED]->(m:Movie)
WHERE r.rating > average
MATCH (m)-[:IS_GENRE]->(g:Genre)<-[:IS_GENRE]-(rm:Movie)
WITH me, m, rm, COUNT(*) AS gs
OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Person)-[:ACTED_IN]->(rm)
WITH me, m, rm, gs, COUNT(a) AS as
OPTIONAL MATCH (m)<-[:DIRECTED]-(d:Person)-[:DIRECTED]->(rm)
WITH me, m, rm, gs, as, COUNT(d) AS ds
MATCH (rm)
WHERE  NOT (me)-[:RATED]->(rm)
RETURN rm.title AS recommendation,
gs as genre_score, as as actor_score, ds as director_score,
(5*gs)+(2*as)+(5*ds) AS weighed_score 
ORDER BY weighed_score DESC LIMIT 10"""
graph.run(query).data()

[{'recommendation': 'The Hunger Games: Mockingjay - Part 2 (2015)',
  'genre_score': 2,
  'actor_score': 52,
  'director_score': 1,
  'weighed_score': 119},
 {'recommendation': 'Grown Ups 2 (2013)',
  'genre_score': 1,
  'actor_score': 52,
  'director_score': 1,
  'weighed_score': 114},
 {'recommendation': 'Toy Story 3 (2010)',
  'genre_score': 5,
  'actor_score': 44,
  'director_score': 0,
  'weighed_score': 113},
 {'recommendation': 'Cloudy with a Chance of Meatballs (2009)',
  'genre_score': 3,
  'actor_score': 44,
  'director_score': 0,
  'weighed_score': 103},
 {'recommendation': 'Kung Fu Panda 3 (2016)',
  'genre_score': 3,
  'actor_score': 44,
  'director_score': 0,
  'weighed_score': 103},
 {'recommendation': 'Toy Story 3 (2010)',
  'genre_score': 3,
  'actor_score': 40,
  'director_score': 0,
  'weighed_score': 95},
 {'recommendation': 'Despicable Me 2 (2013)',
  'genre_score': 3,
  'actor_score': 36,
  'director_score': 0,
  'weighed_score': 87},
 {'recommendation': '22 Jump 

In [41]:
query="""MATCH (me:User{id:'318'})-[r:RATED]-(m:Movie)
WITH me, avg(r.rating) AS average
MATCH (me)-[r:RATED]->(m:Movie)
WHERE r.rating > 4.5
MATCH (m)<-[:ACTED_IN]-(a:Person)-[:ACTED_IN]->(rm)
MATCH (rm)
WHERE  NOT (me)-[:RATED]->(rm)
RETURN rm, a, me ,m LIMIT 50"""
graph.run(query).data()

[{'rm': Node('Movie', id='3990', title='Rugrats in Paris: The Movie (2000)', tmdbId='16340'),
  'a': Node('Person', name='Debbie Reynolds'),
  'me': Node('User', id='318'),
  'm': Node('Movie', id='1884', title='Fear and Loathing in Las Vegas (1998)', tmdbId='1878')},
 {'rm': Node('Movie', id='92264', title='One for the Money (2012)', tmdbId='54054'),
  'a': Node('Person', name='Debbie Reynolds'),
  'me': Node('User', id='318'),
  'm': Node('Movie', id='1884', title='Fear and Loathing in Las Vegas (1998)', tmdbId='1878')},
 {'rm': Node('Movie', id='1614', title='In & Out (1997)', tmdbId='10806'),
  'a': Node('Person', name='Debbie Reynolds'),
  'me': Node('User', id='318'),
  'm': Node('Movie', id='1884', title='Fear and Loathing in Las Vegas (1998)', tmdbId='1878')},
 {'rm': Node('Movie', id='3257', title='Bodyguard, The (1992)', tmdbId='619'),
  'a': Node('Person', name='Debbie Reynolds'),
  'me': Node('User', id='318'),
  'm': Node('Movie', id='1884', title='Fear and Loathing in Las

In [43]:
#Jacard Index
query="""MATCH (me:User{id:'220'})-[r:RATED]-(m:Movie)
WITH me, avg(r.rating) AS mean
MATCH (me)-[r:RATED]->(m:Movie)
WHERE r.rating =5
MATCH (m)-[:ACTED_IN|:DIRECTED]-(t)-[:ACTED_IN|:DIRECTED]-(other:Movie)
WHERE  NOT (me)-[:RATED]->(other)
WITH me, m, other, COUNT(t) AS intersection, COLLECT(t.name) AS i
MATCH (m)-[:ACTED_IN|:DIRECTED]-(mt)
WITH me, m,other, intersection,i, COLLECT(mt.name) AS s1
MATCH (other)-[:ACTED_IN|:DIRECTED]-(ot)
WITH me, m,other,intersection,i, s1, COLLECT(ot.name) AS s2
WITH me, m,other,intersection,s1,s2
WITH me, m,other,intersection,s1+[x IN s2 WHERE NOT x IN s1] AS union, s1, s2
RETURN m.title, other.title, s1,s2,((1.0*intersection)/SIZE(union)) AS jaccard ORDER BY jaccard DESC LIMIT 20"""
graph.run(query).data()

[{'m.title': 'Casablanca (1942)',
  'other.title': 'Top Hat (1935)',
  's1': ['Gino Corrado', 'Gino Corrado'],
  's2': ['Gino Corrado', 'Gino Corrado'],
  'jaccard': 2.0},
 {'m.title': 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)',
  'other.title': "Pirates of the Caribbean: At World's End (2007)",
  's1': ['Gore Verbinski',
   'Angus Barnett',
   'Keira Knightley',
   'Orlando Bloom',
   'Jack Davenport',
   'Mackenzie Crook',
   'Angus Barnett',
   'Martin Klebba',
   'Greg Ellis',
   'Johnny Depp',
   'David Bailie',
   'Zoe Saldana',
   'Jonathan Pryce',
   'Kevin McNally',
   'Geoffrey Rush',
   'Johnny Depp',
   'Martin Klebba',
   'Jack Davenport',
   'Mackenzie Crook',
   'David Bailie',
   'Kevin McNally',
   'Jonathan Pryce',
   'Keira Knightley',
   'Greg Ellis',
   'Geoffrey Rush',
   'Zoe Saldana',
   'Orlando Bloom'],
  's2': ['Gore Verbinski',
   'Johnny Depp',
   'Martin Klebba',
   'Keira Knightley',
   'David Bailie',
   'Jonathan Pryce',
   'Stellan

In [44]:
#Pearson correlation coefficient
query="""MATCH (me:User {id:"220"})-[r:RATED]->(m:Movie)
WITH me, avg(r.rating) AS my_average
MATCH (me)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other)
WITH me, my_average, other, COLLECT({r1: r1, r2: r2}) AS ratings WHERE size(ratings) > 10
MATCH (other)-[r:RATED]->(m:Movie)
WITH me, my_average, other, avg(r.rating) AS other_average, ratings
UNWIND ratings AS r
WITH sum( (r.r1.rating- my_average) * (r.r2.rating- other_average) ) AS a,
sqrt( sum( (r.r1.rating - my_average)^2) * sum( (r.r2.rating - other_average) ^2)) AS b,
me, other 
WHERE b <> 0
RETURN me.id, other.id, a/b as correlation
ORDER BY correlation DESC LIMIT 10"""
graph.run(query).data()

[{'me.id': '220', 'other.id': '494', 'correlation': 0.7825315077845472},
 {'me.id': '220', 'other.id': '32', 'correlation': 0.7818916367269144},
 {'me.id': '220', 'other.id': '485', 'correlation': 0.7633105914491698},
 {'me.id': '220', 'other.id': '97', 'correlation': 0.7547965924537345},
 {'me.id': '220', 'other.id': '79', 'correlation': 0.7399103445131806},
 {'me.id': '220', 'other.id': '88', 'correlation': 0.7328107458190379},
 {'me.id': '220', 'other.id': '124', 'correlation': 0.7185930116701769},
 {'me.id': '220', 'other.id': '235', 'correlation': 0.716539350928329},
 {'me.id': '220', 'other.id': '436', 'correlation': 0.7043763345369647},
 {'me.id': '220', 'other.id': '500', 'correlation': 0.6597414172261901}]

In [45]:
query="""MATCH (me:User {id:"220"})-[:RATED]->(m:Movie)
MATCH (other:User {id:"494"})-[:RATED]->(m:Movie)
RETURN me, other, m"""
graph.run(query).data()

[{'me': Node('User', id='220'),
  'other': Node('User', id='494'),
  'm': Node('Movie', id='858', title='Godfather, The (1972)', tmdbId='238')},
 {'me': Node('User', id='220'),
  'other': Node('User', id='494'),
  'm': Node('Movie', id='589', title='Terminator 2: Judgment Day (1991)', tmdbId='280')},
 {'me': Node('User', id='220'),
  'other': Node('User', id='494'),
  'm': Node('Movie', id='1200', title='Aliens (1986)', tmdbId='679')},
 {'me': Node('User', id='220'),
  'other': Node('User', id='494'),
  'm': Node('Movie', id='1198', title='Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)', tmdbId='85')},
 {'me': Node('User', id='220'),
  'other': Node('User', id='494'),
  'm': Node('Movie', id='260', title='Star Wars: Episode IV - A New Hope (1977)', tmdbId='11')},
 {'me': Node('User', id='220'),
  'other': Node('User', id='494'),
  'm': Node('Movie', id='1210', title='Star Wars: Episode VI - Return of the Jedi (1983)', tmdbId='1892')},
 {'me': Node('User'

In [46]:
query="""MATCH (me:User {id:"220"})-[r:RATED]->(m:Movie)
WITH me, avg(r.rating) AS my_average
MATCH (me)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other)
WITH me, my_average, other, COLLECT({r1: r1, r2: r2}) AS ratings WHERE size(ratings) > 10
MATCH (other)-[r:RATED]->(m:Movie)
WITH me, my_average, other, avg(r.rating) AS other_average, ratings
UNWIND ratings AS r
WITH sum( (r.r1.rating- my_average) * (r.r2.rating- other_average) ) AS a,
sqrt( sum( (r.r1.rating - my_average)^2) * sum( (r.r2.rating - other_average) ^2)) AS b,
me, other 
WHERE b <> 0
WITH me, other, a/b as correlation
ORDER BY correlation DESC LIMIT 10
MATCH (other)-[r:RATED]->(m:Movie) WHERE NOT EXISTS( (me)-[:RATED]->(m) )
WITH m,  SUM( correlation* r.rating) AS score, COLLECT(other) AS other
RETURN m, other, score
ORDER BY score DESC LIMIT 10"""
graph.run(query).data()

[{'m': Node('Movie', id='593', title='Silence of the Lambs, The (1991)', tmdbId='274'),
  'other': [Node('User', id='32'),
   Node('User', id='485'),
   Node('User', id='97'),
   Node('User', id='124'),
   Node('User', id='235'),
   Node('User', id='436')],
  'score': 20.070191908082343},
 {'m': Node('Movie', id='527', title="Schindler's List (1993)", tmdbId='424'),
  'other': [Node('User', id='32'),
   Node('User', id='88'),
   Node('User', id='235'),
   Node('User', id='436')],
  'score': 13.16338795751028},
 {'m': Node('Movie', id='457', title='Fugitive, The (1993)', tmdbId='5503'),
  'other': [Node('User', id='485'),
   Node('User', id='79'),
   Node('User', id='235'),
   Node('User', id='436')],
  'score': 12.400922820247542},
 {'m': Node('Movie', id='110', title='Braveheart (1995)', tmdbId='197'),
  'other': [Node('User', id='494'),
   Node('User', id='485'),
   Node('User', id='124'),
   Node('User', id='235')],
  'score': 11.58382225782918},
 {'m': Node('Movie', id='440', title

In [47]:
query="""MATCH (me:User {id:"220"})-[r:RATED]->(m:Movie)
WITH me, avg(r.rating) AS my_average
MATCH (me)-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other)
WITH me, my_average, other, COLLECT({r1: r1, r2: r2}) AS ratings WHERE size(ratings) > 10
MATCH (other)-[r:RATED]->(m:Movie)
WITH me, my_average, other, avg(r.rating) AS other_average, ratings
UNWIND ratings AS r
WITH sum( (r.r1.rating- my_average) * (r.r2.rating- other_average) ) AS a,
sqrt( sum( (r.r1.rating - my_average)^2) * sum( (r.r2.rating - other_average) ^2)) AS b,
me, other 
WHERE b <> 0
WITH me, other, a/b as correlation
ORDER BY correlation ASC LIMIT 10
MATCH (other)-[r:RATED]->(m:Movie) WHERE NOT EXISTS( (me)-[:RATED]->(m) )
WITH m,  SUM( correlation* r.rating) AS score, COLLECT(other) AS other
RETURN m, other, score
ORDER BY score ASC LIMIT 10"""
graph.run(query).data()

[{'m': Node('Movie', id='4973', title="Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)", tmdbId='194'),
  'other': [Node('User', id='23'),
   Node('User', id='416'),
   Node('User', id='471'),
   Node('User', id='110'),
   Node('User', id='89')],
  'score': -7.210099377998988},
 {'m': Node('Movie', id='527', title="Schindler's List (1993)", tmdbId='424'),
  'other': [Node('User', id='395'),
   Node('User', id='416'),
   Node('User', id='471'),
   Node('User', id='174'),
   Node('User', id='110')],
  'score': -6.143148263336504},
 {'m': Node('Movie', id='750', title='Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', tmdbId='935'),
  'other': [Node('User', id='23'),
   Node('User', id='416'),
   Node('User', id='212'),
   Node('User', id='89')],
  'score': -6.019002146132842},
 {'m': Node('Movie', id='3949', title='Requiem for a Dream (2000)', tmdbId='641'),
  'other': [Node('User', id='23'),
   Node('User', id='212'),
   Node('User', id='110')],
  'score